In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


# 다음의 데이터 경로는 본인 경로에 맞게 수정하시오.

In [ ]:
DATA_PATH = "/content/drive/MyDrive/Colab Notebooks/Data/"

In [ ]:
df = pd.read_csv(f"{DATA_PATH}trans_train.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167465 entries, 0 to 167464
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   고객번호    167465 non-null  int64 
 1   구매일자    167465 non-null  object
 2   지점      167465 non-null  object
 3   상품번호    167465 non-null  int64 
 4   중분류     167465 non-null  object
 5   대분류     167465 non-null  object
 6   구매가격    167465 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 8.9+ MB


In [ ]:
df.head()

,고객번호,구매일자,지점,상품번호,중분류,대분류,구매가격
0,10004,2007-06-26 00:00:00,강남점,234160,영 트렌디,캐주얼,2720000
1,10004,2007-07-08 00:00:00,본 점,165162,4대 B/D,피혁잡화,8550000
2,10004,2007-07-14 00:00:00,본 점,663110,국산냉장고.세탁기,대형가전,28800000
3,10004,2007-07-28 00:00:00,본 점,73119,상품군미지정,기타,81500
4,10004,2007-07-28 00:00:00,본 점,73119,주류,주류,2037500


- datetime 형식으로 변환

In [ ]:
df["구매일자"] = pd.to_datetime(df["구매일자"])

- 날짜끼리 연산이 가능하다.
- 날짜끼리 연산시 Timedelta 타입이 반환된다.
- Timedelta 타입은 Datetime 의 하위 객체로 두개 날짜나 시간의 차이인 기간을 나타낸다.

In [ ]:
time_delta = df["구매일자"].max() - df["구매일자"].min()
time_delta

Timedelta('363 days 00:00:00')

- 일수 뽑기

In [ ]:
time_delta.days

363

- 초 뽑기

In [ ]:
time_delta.total_seconds()

31363200.0

# Feature Engineering

- 새로 만든 feature 와 병합할 고객 번호로만 이루어진 데이터프레임을 만든다.

In [ ]:
features = pd.DataFrame(df["고객번호"].unique(),columns=["고객번호"])
features.head()

,고객번호
0,10004
1,10013
2,10023
3,10040
4,10052


## 구매일자를 이용한 특성생성

- 내점일수
- 구매주기
- 주말방문비율
- 주구매요일
- 봄구매비율
- 여름구매비율
- 가을구매비율
- 겨울구매비율
- 일평균구매건수
- 거래개월수

In [ ]:
df["구매일자"].dt.weekday > 4

0         False
1          True
2          True
3          True
4          True
          ...  
167460    False
167461     True
167462    False
167463    False
167464    False
Name: 구매일자, Length: 167465, dtype: bool

In [ ]:
(df["구매일자"].dt.weekday > 4).mean()

0.29615143462813126

In [ ]:
agg_list = [
    # (컬럼명, 집계방식)
    ("내점일수","nunique"),
    ("구매주기",lambda x : (x.max() - x.min()).days / x.nunique() ), # 최근 구매일과 최초 구매일의 차이를 구하고 내점일수로 나누면 구매주기라고 볼수있음
    ("주말방문비율", lambda x : (x.dt.weekday > 4).mean() ), # 4보다 크면 토,일 이니까!!
    ("주구매요일" , lambda x: x.dt.weekday.mode()[0] ), # 최빈값이 여러개 일수도 있어서..
    ("봄구매비율" , lambda x : (x.dt.month.isin([3,4,5])).mean() ),
    ("여름구매비율" , lambda x : (x.dt.month.isin([6,7,8])).mean() ),
    ("가을구매비율" , lambda x : (x.dt.month.isin([9,10,11])).mean() ),
    ("겨울구매비율" , lambda x : (x.dt.month.isin([12,1,2])).mean() ),
    ("일평균구매건수" , lambda x : x.count() / x.nunique() ), # 구매건수를 구매일자로 나누면 일평균 구매건수라고 볼수있음.
    ("거래개월수" , lambda x : x.dt.month.nunique() )

]

tmp = df.groupby("고객번호")["구매일자"].agg(agg_list).reset_index()
features = features.merge(tmp,how="left",on = "고객번호") # features 변수에 merge 한 결과를 확인하고 변수에 넣으세요!!
features

,고객번호,내점일수,구매주기,주말방문비율,주구매요일,봄구매비율,여름구매비율,가을구매비율,겨울구매비율,일평균구매건수,거래개월수
0,10004,26,11.423077,0.617647,5,0.161765,0.397059,0.250000,0.191176,2.615385,9
1,10013,11,21.090909,0.117647,4,0.352941,0.235294,0.352941,0.058824,1.545455,5
2,10023,21,16.666667,0.348837,3,0.348837,0.162791,0.232558,0.255814,2.047619,11
3,10040,24,13.833333,0.121212,4,0.272727,0.151515,0.318182,0.257576,2.750000,10
4,10052,28,8.892857,0.333333,2,0.585586,0.243243,0.171171,0.000000,3.964286,9
...,...,...,...,...,...,...,...,...,...,...,...
2549,49813,83,4.253012,0.167116,1,0.226415,0.339623,0.237197,0.196765,4.469880,12
2550,49892,31,10.096774,0.363636,6,0.181818,0.289256,0.380165,0.148760,3.903226,11
2551,49925,3,18.333333,0.333333,3,0.000000,1.000000,0.000000,0.000000,1.000000,2
2552,49943,7,30.285714,0.545455,5,0.090909,0.545455,0.272727,0.090909,1.571429,6


## 지점을 이용한 특성생성
- 방문지점수
- 주구매지점

In [ ]:
agg_list = [
    # (컬럼명, 집계방식)
    ("방문지점수","nunique"),
    ("주구매지점" , lambda x : x.mode()[0])

]

tmp = df.groupby("고객번호")["지점"].agg(agg_list).reset_index()
features = features.merge(tmp,how="left",on = "고객번호") # features 변수에 merge 한 결과를 확인하고 변수에 넣으세요!!
features

,고객번호,내점일수,구매주기,주말방문비율,주구매요일,봄구매비율,여름구매비율,가을구매비율,겨울구매비율,일평균구매건수,거래개월수,방문지점수,주구매지점
0,10004,26,11.423077,0.617647,5,0.161765,0.397059,0.250000,0.191176,2.615385,9,3,본 점
1,10013,11,21.090909,0.117647,4,0.352941,0.235294,0.352941,0.058824,1.545455,5,3,본 점
2,10023,21,16.666667,0.348837,3,0.348837,0.162791,0.232558,0.255814,2.047619,11,3,잠실점
3,10040,24,13.833333,0.121212,4,0.272727,0.151515,0.318182,0.257576,2.750000,10,2,부산본점
4,10052,28,8.892857,0.333333,2,0.585586,0.243243,0.171171,0.000000,3.964286,9,4,인천점
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2549,49813,83,4.253012,0.167116,1,0.226415,0.339623,0.237197,0.196765,4.469880,12,1,부산본점
2550,49892,31,10.096774,0.363636,6,0.181818,0.289256,0.380165,0.148760,3.903226,11,2,강남점
2551,49925,3,18.333333,0.333333,3,0.000000,1.000000,0.000000,0.000000,1.000000,2,1,본 점
2552,49943,7,30.285714,0.545455,5,0.090909,0.545455,0.272727,0.090909,1.571429,6,1,본 점


## 상품번호를 이용한 특성생성
- 상품번호수

In [ ]:
agg_list = [
             ('상품번호수', 'nunique'),
             ]

tmp = df.groupby('고객번호')["상품번호"].agg(agg_list).reset_index()

features = features.merge(tmp, how='left') # features 변수에 merge 한 결과를 확인하고 변수에 넣으세요!!

features.head()

,고객번호,내점일수,구매주기,주말방문비율,주구매요일,봄구매비율,여름구매비율,가을구매비율,겨울구매비율,일평균구매건수,거래개월수,방문지점수,주구매지점,상품번호수
0,10004,26,11.423077,0.617647,5,0.161765,0.397059,0.250000,0.191176,2.615385,9,3,본 점,45
1,10013,11,21.090909,0.117647,4,0.352941,0.235294,0.352941,0.058824,1.545455,5,3,본 점,10
2,10023,21,16.666667,0.348837,3,0.348837,0.162791,0.232558,0.255814,2.047619,11,3,잠실점,30
3,10040,24,13.833333,0.121212,4,0.272727,0.151515,0.318182,0.257576,2.750000,10,2,부산본점,34
4,10052,28,8.892857,0.333333,2,0.585586,0.243243,0.171171,0.000000,3.964286,9,4,인천점,57


## 중분류를 이용한 특성생성
- 중분류수
- 주구매중분류

In [ ]:
agg_list = [
            ('중분류수', 'nunique'),
            ('주구매중분류', lambda x: x.mode()[0]), 
            ]

tmp = df.groupby('고객번호')["중분류"].agg(agg_list).reset_index()

features = features.merge(tmp, how='left')  # features 변수에 merge 한 결과를 확인하고 변수에 넣으세요!!

features.head()

,고객번호,내점일수,구매주기,주말방문비율,주구매요일,봄구매비율,여름구매비율,가을구매비율,겨울구매비율,일평균구매건수,거래개월수,방문지점수,주구매지점,상품번호수,중분류수,주구매중분류
0,10004,26,11.423077,0.617647,5,0.161765,0.397059,0.250000,0.191176,2.615385,9,3,본 점,45,39,상품군미지정
1,10013,11,21.090909,0.117647,4,0.352941,0.235294,0.352941,0.058824,1.545455,5,3,본 점,10,12,N.B정장
2,10023,21,16.666667,0.348837,3,0.348837,0.162791,0.232558,0.255814,2.047619,11,3,잠실점,30,23,N/B골프의류
3,10040,24,13.833333,0.121212,4,0.272727,0.151515,0.318182,0.257576,2.750000,10,2,부산본점,34,29,어덜트
4,10052,28,8.892857,0.333333,2,0.585586,0.243243,0.171171,0.000000,3.964286,9,4,인천점,57,37,시티웨어


## 대분류를 이용한 특성생성
- 대분류수
- 주구매대분류

In [ ]:
agg_list = [
            ('대분류수', 'nunique'),
            ('주구매대분류', lambda x: x.mode()[0]),
            ]

tmp = df.groupby('고객번호')["대분류"].agg(agg_list).reset_index()

features = features.merge(tmp, how='left')  # features 변수에 merge 한 결과를 확인하고 변수에 넣으세요!!

features.head()

,고객번호,내점일수,구매주기,주말방문비율,주구매요일,봄구매비율,여름구매비율,가을구매비율,겨울구매비율,일평균구매건수,거래개월수,방문지점수,주구매지점,상품번호수,중분류수,주구매중분류,대분류수,주구매대분류
0,10004,26,11.423077,0.617647,5,0.161765,0.397059,0.250000,0.191176,2.615385,9,3,본 점,45,39,상품군미지정,25,가공식품
1,10013,11,21.090909,0.117647,4,0.352941,0.235294,0.352941,0.058824,1.545455,5,3,본 점,10,12,N.B정장,9,가공식품
2,10023,21,16.666667,0.348837,3,0.348837,0.162791,0.232558,0.255814,2.047619,11,3,잠실점,30,23,N/B골프의류,17,골프
3,10040,24,13.833333,0.121212,4,0.272727,0.151515,0.318182,0.257576,2.750000,10,2,부산본점,34,29,어덜트,15,명품
4,10052,28,8.892857,0.333333,2,0.585586,0.243243,0.171171,0.000000,3.964286,9,4,인천점,57,37,시티웨어,20,시티웨어


## 구매가격을 이용한 특성생성
- 총구매액
- 구매건수
- 평균구매액
- 최대구매액
- 최소구매액
- 환불금액
- 환불건수
- 구매금액표준편차

In [ ]:
agg_list = [
        ('총구매액','sum'),
        ('구매건수', 'count'), 
        ('평균구매액', "mean"),
        ('최대구매액', 'max'),
        ('최소구매액',lambda x: x[x > 0].min() ) ,
        ('환불금액',lambda x: x[x < 0].sum() ) ,
        ('환불건수', lambda x: ( x < 0 ).sum() ), 
        ('구매금액표준편차',lambda x: x[x>0].std() ),
    ]

tmp = df.groupby('고객번호')["구매가격"].agg(agg_list).reset_index()

features = features.merge(tmp, how='left')  # features 변수에 merge 한 결과를 확인하고 변수에 넣으세요!!

features.head()

,고객번호,내점일수,구매주기,주말방문비율,주구매요일,봄구매비율,여름구매비율,가을구매비율,겨울구매비율,일평균구매건수,...,총구매액,구매건수,평균구매액,최대구매액,최소구매액,환불금액,환불건수,구매금액표준편차,내점당 구매건수,월평균구매건수
0,10004,26,11.423077,0.617647,5,0.161765,0.397059,0.250000,0.191176,2.615385,...,390166360,68,5.737741e+06,41760000,6570.0,-4700000,1,9.453819e+06,68,7.555556
1,10013,11,21.090909,0.117647,4,0.352941,0.235294,0.352941,0.058824,1.545455,...,22447800,17,1.320459e+06,13375000,6900.0,0,0,3.179888e+06,17,3.400000
2,10023,21,16.666667,0.348837,3,0.348837,0.162791,0.232558,0.255814,2.047619,...,81765810,43,1.901530e+06,14325000,1250.0,-6003000,1,3.738478e+06,43,3.909091
3,10040,24,13.833333,0.121212,4,0.272727,0.151515,0.318182,0.257576,2.750000,...,213705680,66,3.237965e+06,24738000,1500.0,-12948000,3,5.883481e+06,66,6.600000
4,10052,28,8.892857,0.333333,2,0.585586,0.243243,0.171171,0.000000,3.964286,...,358336570,111,3.228257e+06,38328000,1250.0,-30868000,3,6.959274e+06,111,12.333333


In [ ]:
features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2554 entries, 0 to 2553
Data columns (total 26 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   고객번호      2554 non-null   int64  
 1   내점일수      2554 non-null   int64  
 2   구매주기      2554 non-null   float64
 3   주말방문비율    2554 non-null   float64
 4   주구매요일     2554 non-null   int64  
 5   봄구매비율     2554 non-null   float64
 6   여름구매비율    2554 non-null   float64
 7   가을구매비율    2554 non-null   float64
 8   겨울구매비율    2554 non-null   float64
 9   일평균구매건수   2554 non-null   float64
 10  거래개월수     2554 non-null   int64  
 11  방문지점수     2554 non-null   int64  
 12  주구매지점     2554 non-null   object 
 13  상품번호수     2554 non-null   int64  
 14  중분류수      2554 non-null   int64  
 15  주구매중분류    2554 non-null   object 
 16  대분류수      2554 non-null   int64  
 17  주구매대분류    2554 non-null   object 
 18  총구매액      2554 non-null   int64  
 19  구매건수      2554 non-null   int64  
 20  평균구매액     2554 non-null   floa

- 특성(feature) 생성 과정에서 결측치가 생기는 부분은 신경 쓰지말자
    - 나중에 머신러닝모델에 넣기전에 채우면 된다.
- 범주형 특성을 추가하는 과정에서 인코딩을 어떻게 해야할지도 신경 쓰지말자
    - 나중에 머신러닝모델에 넣기전에 고민해도 된다.
- 결론은 새로운 특성을 추가하는데 집중하자는 말입니다. 우선 특성추가하는데 집중하세요~

# 여기서 부터 features 변수에 특성을 추가해주면 됩니다.
- features 변수에 들어있는 데이터의 행이 줄어들거나 늘어나서는 안됩니다!! 
- 항상 merge 한 결과를 확인하고 features 변수에 할당해주세요.
- 추가한 특성에 대한 간단한 설명 넣어주면 감사!


## VIP 고객 판단
- 평균 구매액의 75% 금액을 정해 이상 사용한 고객만 정의하기


In [ ]:
arr = df["고객번호"].unique()
columns = ["고객번호"]
x_train = pd.DataFrame(arr,columns = columns)

tmp=tmp.astype({"평균구매액":"int"})
#tmp
#x_train
x_train = pd.merge(x_train,tmp,how="left",on="고객번호")
x_train

,고객번호,총구매액,구매건수,평균구매액,최대구매액,최소구매액,환불금액,환불건수,구매금액표준편차
0,10004,390166360,68,5737740,41760000,6570.0,-4700000,1,9.453819e+06
1,10013,22447800,17,1320458,13375000,6900.0,0,0,3.179888e+06
2,10023,81765810,43,1901530,14325000,1250.0,-6003000,1,3.738478e+06
3,10040,213705680,66,3237964,24738000,1500.0,-12948000,3,5.883481e+06
4,10052,358336570,111,3228257,38328000,1250.0,-30868000,3,6.959274e+06
...,...,...,...,...,...,...,...,...,...
2549,49813,47428060,371,127838,9120000,1000.0,-1532000,2,5.931527e+05
2550,49892,54369820,121,449337,7500000,1100.0,0,0,1.148466e+06
2551,49925,1657400,3,552466,1015000,39000.0,0,0,4.899894e+05
2552,49943,40742400,11,3703854,21780000,36000.0,0,0,6.610017e+06


In [ ]:
# 75% 지점 이상치
np.arange(0,1,0.75)
q_arr = tmp["평균구매액"].quantile([0.75]).to_numpy()
q_arr = q_arr.astype(int)
q_arr

array([2758895])

In [ ]:
def chk_con(x,q_arr=2758895):
    if x >= q_arr:
        return "VIP"
    else:
        return "X"

x_train["고객판별"] = x_train["평균구매액"].apply(lambda x : chk_con(x))
x_train

,고객번호,총구매액,구매건수,평균구매액,최대구매액,최소구매액,환불금액,환불건수,구매금액표준편차,고객판별
0,10004,390166360,68,5737740,41760000,6570.0,-4700000,1,9.453819e+06,VIP
1,10013,22447800,17,1320458,13375000,6900.0,0,0,3.179888e+06,X
2,10023,81765810,43,1901530,14325000,1250.0,-6003000,1,3.738478e+06,X
3,10040,213705680,66,3237964,24738000,1500.0,-12948000,3,5.883481e+06,VIP
4,10052,358336570,111,3228257,38328000,1250.0,-30868000,3,6.959274e+06,VIP
...,...,...,...,...,...,...,...,...,...,...
2549,49813,47428060,371,127838,9120000,1000.0,-1532000,2,5.931527e+05,X
2550,49892,54369820,121,449337,7500000,1100.0,0,0,1.148466e+06,X
2551,49925,1657400,3,552466,1015000,39000.0,0,0,4.899894e+05,X
2552,49943,40742400,11,3703854,21780000,36000.0,0,0,6.610017e+06,VIP



## 휴면/이탈 
- 최근 방문일로 부터 지난 시간이, 평균 구매 주기보다 길 경우, 휴면/이탈 가능성이 있는 것으로 고려


In [ ]:
df['구매일자'].min(), df['구매일자'].max() # 데이터 발표일을 유추하여 마지막 데이터 날짜느 2007-12-31이므로, 2008년에 발표했다고 가정하자.(2008-01-01)

(Timestamp('2007-01-02 00:00:00'), Timestamp('2007-12-31 00:00:00'))

In [ ]:
tmp = df.groupby('고객번호')['구매일자'].max() # 최근 방문 일자
tmp

고객번호
10004   2007-10-31
10013   2007-12-01
10023   2007-12-21
10040   2007-12-11
10052   2007-11-25
           ...    
49813   2007-12-26
49892   2007-11-21
49925   2007-07-26
49943   2007-12-17
49974   2007-11-25
Name: 구매일자, Length: 2554, dtype: datetime64[ns]

In [ ]:
features2 = tmp.copy()
features2 = features2.reset_index()
features2

,고객번호,구매일자
0,10004,2007-10-31
1,10013,2007-12-01
2,10023,2007-12-21
3,10040,2007-12-11
4,10052,2007-11-25
...,...,...
2549,49813,2007-12-26
2550,49892,2007-11-21
2551,49925,2007-07-26
2552,49943,2007-12-17


In [ ]:
avg_day = features.groupby('고객번호')['구매주기'].mean()
avg_day.reset_index()

,고객번호,구매주기
0,10004,11.423077
1,10013,21.090909
2,10023,16.666667
3,10040,13.833333
4,10052,8.892857
...,...,...
2549,49813,4.253012
2550,49892,10.096774
2551,49925,18.333333
2552,49943,30.285714


In [ ]:
features2 = pd.merge(features2,avg_day,how="left",on="고객번호")
features2

,고객번호,구매일자,구매주기
0,10004,2007-10-31,11.423077
1,10013,2007-12-01,21.090909
2,10023,2007-12-21,16.666667
3,10040,2007-12-11,13.833333
4,10052,2007-11-25,8.892857
...,...,...,...
2549,49813,2007-12-26,4.253012
2550,49892,2007-11-21,10.096774
2551,49925,2007-07-26,18.333333
2552,49943,2007-12-17,30.285714


In [ ]:
# 2008년 발표날과 최근 구매일의 차이
from datetime import datetime
df["구매일자"] = pd.to_datetime(df["구매일자"])
mk_dt = datetime(2008,1,1)
time_delta = mk_dt - features2["구매일자"]
td = time_delta.dt.days # timedelta 타입 변환

td

0        62
1        31
2        11
3        21
4        37
       ... 
2549      6
2550     41
2551    159
2552     15
2553     37
Name: 구매일자, Length: 2554, dtype: int64

In [ ]:
idx= pd.DataFrame(df["고객번호"].unique(),columns=["고객번호"])

idx = pd.concat([idx,td], axis=1)

In [ ]:
x = pd.merge(features2,idx, how="left",on="고객번호")
x

,고객번호,구매일자_x,구매주기,구매일자_y
0,10004,2007-10-31,11.423077,62
1,10013,2007-12-01,21.090909,31
2,10023,2007-12-21,16.666667,11
3,10040,2007-12-11,13.833333,21
4,10052,2007-11-25,8.892857,37
...,...,...,...,...
2549,49813,2007-12-26,4.253012,6
2550,49892,2007-11-21,10.096774,41
2551,49925,2007-07-26,18.333333,159
2552,49943,2007-12-17,30.285714,15


In [ ]:
# 휴면고객인지 아닌지의 판별치
sub1=x["구매주기"]/x["구매일자_y"]
sub1 = pd.DataFrame(sub1)
sub1

,0
0,0.184243
1,0.680352
2,1.515152
3,0.658730
4,0.240347
...,...
2549,0.708835
2550,0.246263
2551,0.115304
2552,2.019048


In [ ]:
x = (sub1 >= 1).astype(int).rename(columns={0:"휴면/이탈 고객 여부"})
x = pd.DataFrame(x)
x

,휴면/이탈 고객 여부
0,0
1,0
2,1
3,0
4,0
...,...
2549,0
2550,0
2551,0
2552,1


In [ ]:
a = pd.DataFrame(df["고객번호"].unique(),columns=["고객번호"])
y_train = pd.concat([a,x], axis=1)
y_train

,고객번호,휴면/이탈 고객 여부
0,10004,0
1,10013,0
2,10023,1
3,10040,0
4,10052,0
...,...,...
2549,49813,0
2550,49892,0
2551,49925,0
2552,49943,1


In [ ]:
features = features.merge(y_train, how='left')  # features 변수에 merge 한 결과를 확인하고 변수에 넣으세요!!
features.head()

,고객번호,내점일수,구매주기,주말방문비율,주구매요일,봄구매비율,여름구매비율,가을구매비율,겨울구매비율,일평균구매건수,...,구매건수,평균구매액,최대구매액,최소구매액,환불금액,환불건수,구매금액표준편차,내점당 구매건수,월평균구매건수,휴면/이탈 고객 여부
0,10004,26,11.423077,0.617647,5,0.161765,0.397059,0.250000,0.191176,2.615385,...,68,5.737741e+06,41760000,6570.0,-4700000,1,9.453819e+06,68,7.555556,0
1,10013,11,21.090909,0.117647,4,0.352941,0.235294,0.352941,0.058824,1.545455,...,17,1.320459e+06,13375000,6900.0,0,0,3.179888e+06,17,3.400000,0
2,10023,21,16.666667,0.348837,3,0.348837,0.162791,0.232558,0.255814,2.047619,...,43,1.901530e+06,14325000,1250.0,-6003000,1,3.738478e+06,43,3.909091,1
3,10040,24,13.833333,0.121212,4,0.272727,0.151515,0.318182,0.257576,2.750000,...,66,3.237965e+06,24738000,1500.0,-12948000,3,5.883481e+06,66,6.600000,0
4,10052,28,8.892857,0.333333,2,0.585586,0.243243,0.171171,0.000000,3.964286,...,111,3.228257e+06,38328000,1250.0,-30868000,3,6.959274e+06,111,12.333333,0



## 월평균 구매빈도


In [ ]:
agg_list = [
    ("월평균구매빈도", lambda x : x.count() / x.dt.month.nunique() )
]
tmp = df.groupby("고객번호")["구매일자"].agg(agg_list).reset_index()
tmp

,고객번호,월평균구매빈도
0,10004,7.555556
1,10013,3.400000
2,10023,3.909091
3,10040,6.600000
4,10052,12.333333
...,...,...
2549,49813,30.916667
2550,49892,11.000000
2551,49925,1.500000
2552,49943,1.833333


In [ ]:
features = features.merge(tmp,how="left",on = "고객번호") 
features

,고객번호,내점일수,구매주기,주말방문비율,주구매요일,봄구매비율,여름구매비율,가을구매비율,겨울구매비율,일평균구매건수,...,평균구매액,최대구매액,최소구매액,환불금액,환불건수,구매금액표준편차,내점당 구매건수,월평균구매건수,휴면/이탈 고객 여부,월평균구매빈도
0,10004,26,11.423077,0.617647,5,0.161765,0.397059,0.250000,0.191176,2.615385,...,5.737741e+06,41760000,6570.0,-4700000,1,9.453819e+06,68,7.555556,0,7.555556
1,10013,11,21.090909,0.117647,4,0.352941,0.235294,0.352941,0.058824,1.545455,...,1.320459e+06,13375000,6900.0,0,0,3.179888e+06,17,3.400000,0,3.400000
2,10023,21,16.666667,0.348837,3,0.348837,0.162791,0.232558,0.255814,2.047619,...,1.901530e+06,14325000,1250.0,-6003000,1,3.738478e+06,43,3.909091,1,3.909091
3,10040,24,13.833333,0.121212,4,0.272727,0.151515,0.318182,0.257576,2.750000,...,3.237965e+06,24738000,1500.0,-12948000,3,5.883481e+06,66,6.600000,0,6.600000
4,10052,28,8.892857,0.333333,2,0.585586,0.243243,0.171171,0.000000,3.964286,...,3.228257e+06,38328000,1250.0,-30868000,3,6.959274e+06,111,12.333333,0,12.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2549,49813,83,4.253012,0.167116,1,0.226415,0.339623,0.237197,0.196765,4.469880,...,1.278384e+05,9120000,1000.0,-1532000,2,5.931527e+05,371,30.916667,0,30.916667
2550,49892,31,10.096774,0.363636,6,0.181818,0.289256,0.380165,0.148760,3.903226,...,4.493374e+05,7500000,1100.0,0,0,1.148466e+06,121,11.000000,0,11.000000
2551,49925,3,18.333333,0.333333,3,0.000000,1.000000,0.000000,0.000000,1.000000,...,5.524667e+05,1015000,39000.0,0,0,4.899894e+05,3,1.500000,0,1.500000
2552,49943,7,30.285714,0.545455,5,0.090909,0.545455,0.272727,0.090909,1.571429,...,3.703855e+06,21780000,36000.0,0,0,6.610017e+06,11,1.833333,1,1.833333


### 중분류 특성을 이용해서 지점베스트품목 만들기

In [ ]:
lst = df["중분류"].value_counts().index.tolist()

In [ ]:
agg_list = [
    ("지점베스트품목", lambda x : x[x.isin(lst)].unique())
]
df.groupby('고객번호')["중분류"].agg(agg_list).reset_index()

,고객번호,지점베스트품목
0,10004,"[영 트렌디, 4대 B/D, 국산냉장고.세탁기, 상품군미지정, 주류, N.B, 장갑..."
1,10013,"[커피숍, NB 여화, N.B정장, 수영복, 선식(가루류), 비지니스군, 전기밥솥,..."
2,10023,"[일식델리, 유제품, 상품군미지정, 우육, 양념육, N/B골프의류, 디자이너부틱, ..."
3,10040,"[어덜트, 기초 화장품, 상품군미지정, 시티캐주얼, 유아용품, 햄, 생선, 농산가공..."
4,10052,"[그라스, 청과, takeout양과, 상품군미지정, N.B, 산지통합, 수 ..."
...,...,...
2549,49813,"[면류, 애견용품, 상품군미지정, 브랜드샵, 돈육, 수예소품, 비지니스군, 넥타이(..."
2550,49892,"[농산가공, 돈육, 일반가공식품, L.B, 햄, 상품군미지정, 어묵, 유기농야채, ..."
2551,49925,"[명품잡화보석, 어덜트]"
2552,49943,"[색조 화장품, 건과, 상품군미지정, 건생선, 시티웨어, 디자이너캐릭터, 기초 화장..."
